In [1]:
import pandas as pd

# import ggplot with letsplot
from lets_plot import *

import sqlite3

LetsPlot.setup_html()

In [2]:
# Creating variables to call each dataframe 'MobyGames.db'
conn = sqlite3.connect('../data/MobyGames.db')

query_games = "SELECT * FROM all_games"
all_games = pd.read_sql_query(query_games, conn)

query_genre = "SELECT * FROM genre_info"
genre_info = pd.read_sql_query(query_genre, conn)

query_release = "SELECT * FROM release_info"
release_info = pd.read_sql_query(query_release, conn)

query_score = "SELECT * FROM score_info"
score_info = pd.read_sql_query(query_score, conn)

All plot analysis is done in [Index.mb](../docs/index.md)

## Plot 1 - Games per Genre

In [3]:
# Split the 'Genre' column by commas and explode it to create a row for each genre
plot1_df = genre_info.assign(Genre=genre_info.Genre.str.split(',')).explode('Genre')

# Remove leading and trailing white spaces from the exploded 'Genre' column
plot1_df['Genre'] = plot1_df['Genre'].str.strip()


In [4]:
count_of_genres = (
    ggplot(data=plot1_df, 
           mapping=aes(x='Genre', fill='Genre')) + 
    geom_bar(alpha=0.5) +
    labs(title='Distribution of Game Genres',
         subtitle='Number of games in each genre for the first Xbox',
         x='Genre',
         y='Number of games') +
    theme_minimal() +
    theme(legend_position='none')
)
count_of_genres

In [5]:
# Saves the plot
ggsave(count_of_genres, '../docs/figures/genre_distribution.svg', path='.', iframe=False)


'/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/docs/figures/genre_distribution.svg'

The figure above shows us the most popular genres for the xbox 

## Plot 2 - Average Score per Genre

In [6]:
# Joins the 'plot1_df' and 'score_info' dataframes on the 'title' column
plot2_df = plot1_df.merge(score_info, on='title')

# Group by 'Genre' and calulate the average score for each genre
plot2_df = plot2_df.groupby('Genre').agg({'average_score':'mean'}).reset_index()

# Sort the dataframe by 'average_score' in descending order
plot2_df = plot2_df.sort_values(by='average_score', ascending=False)

average_score_per_genre = (
    ggplot(data=plot2_df, mapping=aes(x='Genre', y='average_score', fill='Genre')) +
    geom_bar(stat='identity', alpha=0.75) +
    labs(title='Average score per genre', x='Genre', y='Average score') +
    ylim(60, 80) +
    theme_minimal() +
    theme(legend_position='none')
)
average_score_per_genre

In [7]:
ggsave(average_score_per_genre, '../docs/figures/average_score_per_genre.svg', path='.', iframe=False)


'/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/docs/figures/average_score_per_genre.svg'

The figure above shows us the most successful genres for the xbox 

## Plot 3 - Number of Games Released per Developer

In [8]:
# Some entries in 'developes' have multiple values in them, each developer is enclosed with '' when there is more than one. This creates a new row for each developer
plot3_df = release_info.assign(developers=release_info.developers.str.split("',")).explode('developers')

# Strip all the white spaces from the 'Perspective' column again and convert to lowercase
plot3_df['developers'] = plot3_df['developers'].str.strip().str.lower()

# Create a table showing how many games each developer has released
plot3_df = plot3_df.groupby('developers').agg({'title':'count'}).reset_index()
plot3_df = plot3_df.sort_values(by='title', ascending=False)


In [9]:
# Create a visual showing how many games each developer has released, no x axis
games_per_developer = (
    ggplot(data=plot3_df.head(20), mapping=aes(x='developers', y='title', fill='developers')) +
    geom_bar(stat='identity', alpha=0.75) +
    labs(title='Number of games per developer', x='Developer', y='Number of games') +
    theme_minimal() +
    theme(axis_text_x=element_blank()) +
    ggsize(700, 600) 
)
games_per_developer

In [10]:
ggsave(games_per_developer, '../docs/figures/games_per_developer.svg', path='.', iframe=False)

'/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/docs/figures/games_per_developer.svg'

Shows us the number of gmaes for the top 20 xbox developers 

## Plot 4 - Average Score per Game and Number of People Who Worked on Them

In [11]:
# Create a table for the average score per game and the credits of the game
plot4_df = score_info.merge(release_info, on='title')
plot4_df = plot4_df.groupby('title').agg({'average_score':'mean', 'credits':'sum', 'xbox_score':'sum'}).reset_index()
plot4_df = plot4_df.sort_values(by='average_score', ascending=False)


In [12]:
score_per_game_and_workers = (
    ggplot(plot4_df, aes(x='credits', y='average_score', color='xbox_score', label='title')) +
    geom_point(size=3) +
    geom_smooth(method='lm', se=False, linetype='dashed', color='blue') +
    labs(
        title='Average score per game and credits',
        subtitle='Average score of the game and the number of people who worked on it',
        x='Number of people who worked on the game',
        y='Average score',
        color='Xbox score'
    ) +
    theme_minimal() +
    theme(
        plot_title=element_text(hjust=0.5),
        plot_subtitle=element_text(hjust=0.5)
    )
)
score_per_game_and_workers

In [13]:
# Save the plot as an image
ggsave(score_per_game_and_workers, '../docs/figures/score_per_game_and_workers.svg', path='.', iframe=False)

'/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/docs/figures/score_per_game_and_workers.svg'

Shows the relationship between game score and the number of people who worked on the game

## Plot 5 - Comparing three different scores, 'moby_score', 'player_score', and 'critics_score'

In [14]:
# Create a table for the average score per game and the credits of the game
plot5_df = score_info.merge(release_info, on='title')
plot5_df = plot5_df.groupby('title').agg({'moby_score':'mean', 'player_score':'mean', 'critics_score':'mean'}).reset_index()
plot5_df = plot5_df.sort_values(by='moby_score', ascending=False)
#drop the title column
plot5_df = plot5_df.drop('title', axis=1)

In [15]:
# Reshape the dataframe to long format so that the scores are in one column and the score type is in another
plot5_df_long = pd.melt(plot5_df, var_name='score_type', value_name='score')

# Create the plot
score_distribution = (
    ggplot(data=plot5_df_long, mapping=aes(x='score', fill='score_type')) +
    geom_density(alpha=0.7) +
    scale_fill_manual(values={'moby_score': 'lightblue', 'player_score': 'red', 'critics_score': 'green'}) +
    labs(title='Distribution of scores', x='Score', y='Density', fill='Score Type') +
    theme_minimal()     +
    ggsize(900, 400) 
)

In [16]:
ggsave(score_distribution, '../docs/figures/score_distribution.svg', path='.', iframe=False)

'/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/docs/figures/score_distribution.svg'

Shows the distribution of scores for each score giver, 'moby_score', 'critics_score', and 'player_score'